# Final Project: Functional Programming

**Laura Belizón Merchán** - 100452273  
**Jorge Lázaro Ruiz** - 100452172  
*Degree in Applied Mathematics and Computation*

## Brief summary

*Pending...*

In [1]:
-- Necessary imports
import Data.Maybe (fromJust)
import Data.List (intercalate, find)
import Data.Time
import Data.Time.Calendar.WeekDate

## Types

> 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data, newtype) for each of them.

An agenda is described as a list of events, so it makes sense to center the type definitions around the characteristics of an event.

For this, we can create the different types of descriptors defined and then use them for the constructor of an event.

In [2]:
type Name = String
data Type = Personal | Health | Work | Management
data Date = Date {day :: Int, month :: String, year :: Int, workday :: Bool}
data Time = Time {start :: (Int, Int), end :: (Int, Int), duration :: Maybe Int}
newtype Participants = Participants [Name]
data Repetitions = Punctual | Daily Int | Weekday Int | Weekly Int

* `Name` is just an alias for a `String`, as there are no spectial requirements for it.
* `Type` and `Repetitions` need to be instances of class `Show` if we want to be able to print them, and since they can only take a set of values we must define them as a `data`.
* `Date` and `Time` require a constructor with multiple parameters, hence we use `data` to define them. They also have to be instances of class `Show` so we can pretty-print them.
* `Participants` could be defined as an alias for `[Name]` but in order to avoid confusion with just any other list of strings, we decided to make it into a `newtype` to distinguish it and prevent unintended operations with other lists of strings.

Because we will need to display this to the user, we declare each type as a `Show` instance and define `show` for each one.

In [3]:
instance Show Type where
    show Personal = "Personal"
    show Health = "Health"
    show Work = "Work"
    show Management = "Management"

instance Show Date where
    show (Date day month year workday) = show day ++ " " ++ month ++ " " ++ show year ++ " " ++
        if workday then "(Workday)" else ""

leadingZero :: Int -> String
leadingZero n = if n < 10 then "0" ++ show n else show n

instance Show Time where
    show (Time (startHour, startMinute) (endHour, endMinute) duration) = 
        show startHour ++ ":" ++ leadingZero startMinute ++ " - " ++ 
        show endHour ++ ":" ++ leadingZero endMinute ++ 
        maybe "" (\d -> "  (Duration: " ++ show d ++ " minutes)") duration

instance Show Participants where
    show (Participants names) = intercalate ", " names

instance Show Repetitions where
    show Punctual = "Punctual"
    show (Daily n) = "Daily " ++ show n
    show (Weekday n) = "Weekday " ++ show n
    show (Weekly n) = "Weekly " ++ show n

Because we will need to compare descriptors, we declare each one as an `Eq` instance and define `==` for each one.

In [4]:
instance Eq Type where
    Personal == Personal = True
    Health == Health = True
    Work == Work = True
    Management == Management = True
    _ == _ = False

instance Eq Date where
    (Date day1 month1 year1 _) == (Date day2 month2 year2 _) = 
        day1 == day2 && month1 == month2 && year1 == year2

instance Eq Time where
    (Time (startHour1, startMinute1) (endHour1, endMinute1) _) == 
        (Time (startHour2, startMinute2) (endHour2, endMinute2) _) = 
            startHour1 == startHour2 && startMinute1 == startMinute2 && 
            endHour1 == endHour2 && endMinute1 == endMinute2

instance Eq Participants where
    (Participants names1) == (Participants names2) = names1 == names2

instance Eq Repetitions where
    Punctual == Punctual = True
    (Daily n1) == (Daily n2) = n1 == n2
    (Weekday n1) == (Weekday n2) = n1 == n2
    (Weekly n1) == (Weekly n2) = n1 == n2
    _ == _ = False

With all the types defined, all we need to do is create the type `Descriptor`, with a constructor for each field.  
After that, we can define `Event` as a list of `Descriptor`s and `Agenda` as a list of `Event`s.

In [5]:
data Descriptor =
    NameDescriptor Name
    | TypeDescriptor Type
    | DateDescriptor Date
    | TimeDescriptor Time
    | ParticipantsDescriptor Participants
    | RepetitionsDescriptor Repetitions
    deriving Show

type Event = [Descriptor]
type Agenda = [Event]

In [6]:
instance Eq Descriptor where
    (NameDescriptor name1) == (NameDescriptor name2) = name1 == name2
    (TypeDescriptor type1) == (TypeDescriptor type2) = type1 == type2
    (DateDescriptor date1) == (DateDescriptor date2) = date1 == date2
    (TimeDescriptor time1) == (TimeDescriptor time2) = time1 == time2
    (ParticipantsDescriptor participants1) == (ParticipantsDescriptor participants2) = 
        participants1 == participants2
    (RepetitionsDescriptor repetitions1) == (RepetitionsDescriptor repetitions2) = 
        repetitions1 == repetitions2
    _ == _ = False

## Event display

> 2. A function to show the events in a pretty way, following the order of descriptors as presented above.

Because the `Event` type is just a list of `Descriptor`s with no particular order, we need to sort it before printing it.

For this, we created a set of "getter" functions that find instances of a given descriptor in a list of descriptors.

In [7]:
getName :: Event -> Name
getName event = case find isNameDescriptor event of
        Just (NameDescriptor name) -> name
        _ -> error "Event has no name"
    where
        isNameDescriptor (NameDescriptor _) = True
        isNameDescriptor _ = False

getType :: Event -> Type
getType event = case find isTypeDescriptor event of
        Just (TypeDescriptor type') -> type'
        _ -> error "Event has no type"
    where
        isTypeDescriptor (TypeDescriptor _) = True
        isTypeDescriptor _ = False

getDate :: Event -> Maybe Date
getDate event = case find isDateDescriptor event of
        Just (DateDescriptor date) -> Just date
        _ -> Nothing
    where
        isDateDescriptor (DateDescriptor _) = True
        isDateDescriptor _ = False

getTime :: Event -> Maybe Time
getTime event = case find isTimeDescriptor event of
        Just (TimeDescriptor time) -> Just time
        _ -> Nothing
    where
        isTimeDescriptor (TimeDescriptor _) = True
        isTimeDescriptor _ = False

getParticipants :: Event -> Participants
getParticipants event = case find isParticipantsDescriptor event of
        Just (ParticipantsDescriptor participants) -> participants
        _ -> error "Event has no participants"
    where
        isParticipantsDescriptor (ParticipantsDescriptor _) = True
        isParticipantsDescriptor _ = False

getRepetitions :: Event -> Repetitions
getRepetitions event = case find isRepetitionsDescriptor event of
        Just (RepetitionsDescriptor repetitions) -> repetitions
        _ -> error "Event has no repetitions descriptor"
    where
        isRepetitionsDescriptor (RepetitionsDescriptor _) = True
        isRepetitionsDescriptor _ = False

In [8]:
printEvent :: Event -> IO ()
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ show (getName event)
    putStrLn $ "  Type: " ++ show (getType event)
    putStrLn $ "  Date: " ++ maybe "Unknown" show (getDate event)
    putStrLn $ "  Time: " ++ maybe "Unknown" show (getTime event)
    putStrLn $ "  Participants: " ++ show (getParticipants event)
    putStrLn $ "  Repetitions: " ++ show (getRepetitions event)

-- Example usage
eventName = NameDescriptor "Example"
eventType = TypeDescriptor Work
eventDate = DateDescriptor (Date 6 "December" 2023 True)
eventTime = TimeDescriptor (Time (10, 30) (12, 0) (Just 90))
eventParticipants = ParticipantsDescriptor (Participants ["John", "Alice", "Bob"])
eventRepetitions = RepetitionsDescriptor (Weekly 2)
exampleEvent = [eventName, eventType, eventDate, eventTime, eventParticipants, eventRepetitions]
printEvent exampleEvent

Event:
  Name: "Example"
  Type: Work
  Date: 6 December 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

## Utility functions

> 3. An `isWorkingDate Date [Date]` function that receives a `Date` and a list of holidays and returns if the day is weekday or weekend/holiday by considering leap years and the fact that 1<sup>st</sup> January 2022 was Saturday.

To do this, we can use `Data.Time.Calendar`'s `dayOfWeek` function to get the day of the week of a given date.

Since our `Date` type is different from the `Day` in the module, we need to convert it using a custom `toDay` function.

Then, we can check if the date is a Saturday or Sunday, and if it is, return `False`.

In [9]:
toDay :: Date -> Day
toDay (Date d m y _) = fromGregorian (toInteger y) (monthToInt m) d

monthToInt :: String -> Int
monthToInt month = fromJust $ lookup month (zip months [1..12])
    where months = ["January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December"]

isWeekend :: Date -> Bool
isWeekend date =
  let (_, _, dayOfWeek) = toWeekDate $ toDay date
  in dayOfWeek == 6 || dayOfWeek == 7  -- 6 is Saturday, 7 is Sunday

isWorkingDate :: Date -> [Date] -> Bool
isWorkingDate date holidays
    | date `elem` holidays = False
    | otherwise = not $ isWeekend date

-- Example usage
exampleDate = Date 25 "December" 2023 True
exampleHolidays = [Date 25 "December" 2023 False, Date 1 "January" 2024 False]
print $ isWorkingDate exampleDate exampleHolidays


False

> 4. A `searchEvents [Descriptor] Agenda` function that returns all the events of the `Agenda` with such list of `Descriptor`s. It should be able for example to return  events  that  start  at  a  given  time,  with  a  given  frequency,  involving  some persons, etc.

A simple `filter` function can be used to get all the events that match a given list of descriptors.

In [10]:
searchEvents :: [Descriptor] -> Agenda -> [Event]
searchEvents descriptors = filter (matches descriptors)
    where
        matches :: [Descriptor] -> Event -> Bool
        matches descriptors event = all (`elem` event) descriptors

This is not requested, but a function that prints the results in a fancy way is useful.

In [11]:
printEventList :: [Event] -> IO ()
printEventList events = do
    putStrLn $ "Found " ++ show (length events) ++ " events:"
    mapM_ printEvent events

-- Example usage
exampleAgenda = [[NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 2", TypeDescriptor Work, DateDescriptor (Date 2 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 3", TypeDescriptor Work, DateDescriptor (Date 3 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 4", TypeDescriptor Work, DateDescriptor (Date 4 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 5", TypeDescriptor Work, DateDescriptor (Date 5 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 6", TypeDescriptor Work, DateDescriptor (Date 6 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]]
result = searchEvents [TypeDescriptor Work] exampleAgenda
printEventList result

Found 5 events:
Event:
  Name: "Event 2"
  Type: Work
  Date: 2 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 3"
  Type: Work
  Date: 3 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 4"
  Type: Work
  Date: 4 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 5"
  Type: Work
  Date: 5 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 6"
  Type: Work
  Date: 6 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

> 5. A `deleteEvents [Descriptor] Agenda` function that returns an `Agenda` where the events having those descriptors are removed.

Again, using `filter` it's possible to search for events that match a `Descriptor` list.
The `matches` function checks if all the descriptors are present in the event.

In [14]:
deleteEvents :: [Descriptor] -> Agenda -> Agenda
deleteEvents descriptors = filter (not . matches descriptors)
    where
        matches :: [Descriptor] -> Event -> Bool
        matches descriptors event = all (`elem` event) descriptors

-- Example usage
result = deleteEvents [TypeDescriptor Work] exampleAgenda
printEventList result

Found 1 events:
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

[[NameDescriptor "Event 1",TypeDescriptor Health,DateDescriptor 1 January 2023 (Workday),TimeDescriptor 10:30 - 12:00  (Duration: 90 minutes),ParticipantsDescriptor John, Alice, Bob,RepetitionsDescriptor Weekly 2]]

> 6. A `completeTime Event` function that if the `Event` has not `duration` inserts it by considering the `start` and `end` hours. If it has `start` and duration it will insert the end and vice versa. If it has just duration it will do  nothing, as the `Event` has not been scheduled yet. Assume no event will spawn over different days.

In [13]:
completeTime :: Event -> Event
completeTime event = case getTime event of
        Just time@(Time (startHour, startMinute) (endHour, endMinute) Nothing) -> 
            cleanEvent ++ [TimeDescriptor (Time (startHour, startMinute) (endHour, endMinute) (Just duration))]
            where
                duration = (endHour - startHour) * 60 + (endMinute - startMinute)
                cleanEvent = filter (/= TimeDescriptor time) event
        _ -> event

-- Example usage
exampleEvent = [NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (10, 30) (12, 0) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEvent exampleEvent
printEvent $ completeTime exampleEvent

Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00  (Duration: 90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2